In [1]:
%matplotlib ipympl
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.signal import welch, periodogram
import os
import matplotlib.dates as mdates
import utide

from physoce import tseries as ts
from tseries import lombscargle
from scipy.stats import chi2

from airsea.windstress import stress
from oceans import cor_beta
from glob import glob
import gsw
import os.path

### Station M location

In [2]:
mlat = 35+8.4585/60  
mlon = -122-59.9036/60

### Load data

#### NCEP-NARR

In [3]:
narr_dir = '/Users/tomconnolly/work/Data/NCEP-NARR/'

u_file_list = sorted(glob(os.path.join(narr_dir,'uwnd.10m.*.nc')))
v_file_list = sorted(glob(os.path.join(narr_dir,'vwnd.10m.*.nc')))

uds = xr.open_mfdataset(u_file_list)
vds = xr.open_mfdataset(v_file_list)

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/xarray/conventions.py:362: SerializationWarning: variable 'uwnd' has multiple fill values {9.96921e+36, -9.96921e+36}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/xarray/conventions.py:362: SerializationWarning: variable 'uwnd' has multiple fill values {9.96921e+36, -9.96921e+36}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/xarray/conventions.py:362: SerializationWarning: variable 'uwnd' has multiple fill values {9.96921e+36, -9.96921e+36}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/xarray/conventions.py:362: SerializationWarning: variable 'uwnd' has multiple fill values {9.96921e+36, -9.96921e+36}, decoding all values to NaN.
  stack_char_

#### Find NCEP-NARR indices of nearest point to Station M

In [4]:
dist = np.array(np.sqrt(((mlat-uds['lat'])*111)**2 + 
                        ((mlon-uds['lon'])*np.cos(mlat*np.pi/180)*111)**2))

In [5]:
idx = np.argmin(dist,axis=None)
ii,jj = np.unravel_index(idx,np.shape(dist))

#### Compute wind stress curl

In [6]:
dx = 0.5*np.array(vds['x'][jj+1] - vds['x'][jj-1])
dy = 0.5*np.array(uds['y'][ii+1] - uds['y'][ii-1])

f = np.array(gsw.f(uds['lat']))
rho = 1045

beta = np.array(cor_beta(uds['lat'][ii,jj]))
H = 4000

Forcing term in Koblinsky et al.

In [7]:
ext = 2 # length of subset on either side of center
ci = ext  # center index

In [8]:
uwndsub = np.array(uds['uwnd'][:,np.arange(ii-ext,ii+ext+1),np.arange(jj-ext,jj+ext+1)])
vwndsub = np.array(vds['vwnd'][:,np.arange(ii-ext,ii+ext+1),np.arange(jj-ext,jj+ext+1)])

In [9]:
fsub = f[slice(ii-ext,ii+ext+1),slice(jj-ext,jj+ext+1)]

In [10]:
taux = stress(uwndsub)
tauy = stress(vwndsub)

In [11]:
curl_tau_F_2 = fsub[ci,ci]*((1/(2*dx))*(tauy[:,ci,ci+1]/(rho*fsub[ci,ci+1]) - 
                           tauy[:,ci,ci-1]/(rho*fsub[ci,ci-1])) - 
           (1/(2*dy))*(taux[:,ci+1,ci]/(rho*fsub[ci+1,ci]) - 
                   taux[:,ci-1,ci]/(rho*fsub[ci-1,ci])))

In [12]:
curl_tau_F = fsub[ci,ci]*((1/(12*dx))*(-tauy[:,ci,ci+2]/(rho*fsub[ci,ci+2]) + 
                                         8*tauy[:,ci,ci+1]/(rho*fsub[ci,ci+1]) - 
                                         8*tauy[:,ci,ci-1]/(rho*fsub[ci,ci-1]) +
                                         tauy[:,ci,ci-2]/(rho*fsub[ci,ci-2])) -
           (1/(12*dy))*(-taux[:,ci+2,ci]/(rho*fsub[ci+2,ci]) +
                        8*taux[:,ci+1,ci]/(rho*fsub[ci+1,ci]) -
                        8*taux[:,ci-1,ci]/(rho*fsub[ci-1,ci]) +
                        taux[:,ci-2,ci]/(rho*fsub[ci-2,ci])))

In [15]:
plt.figure()
plt.plot(curl_tau_F_2,curl_tau_F)

FigureCanvasNbAgg()

In [18]:
curl_time = uds['time']

In [19]:
t1 = np.datetime64('2015-11-11T12:00')
t2 = np.datetime64('2018-10-14T06:00')

fi, = np.where((curl_time >= t1) & (curl_time <= t2))

#### Rover

In [20]:
# Load data
rover_csv_file = 'data/Rover_II_Current_Mag_Hourly_Avg_pad_2018.csv'
df_all = pd.read_csv(rover_csv_file,parse_dates=[7])

# Datetime index
df_all = df_all.set_index('Date_time_R')

# Create u and v components with units m/s
df_all['u'] = df_all['Easting (cm/sec)']/100
df_all['v'] = df_all['Northing (cm/sec)']/100

# mask erroneous points 
# abrupt shift near end of deployment 
bi = ((df_all.index > np.datetime64('2015-06-18T12:00')) &
    (df_all.index < np.datetime64('2015-06-20T00:00')))

df_all['u'][bi] = np.nan
df_all['v'][bi] = np.nan

# select time range
t1 = np.datetime64('2014-10-11T00:00:00')
t2 = np.datetime64('2018-10-17T00:00:00')
df_all = df_all.loc[(df_all.index >= t1) & (df_all.index <= t2)]

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [53]:
# tidal analysis
time = mdates.date2num(df_all.index)

coef = utide.solve(time, np.array(df_all['u']), 
         np.array(df_all['v']),
         lat=35+8.4585/60,
         method='robust')

solve: matrix prep ... solution ... diagnostics ... done.


In [22]:
coef['Lsmaj']

array([1.55186303e-02, 7.52297995e-03, 6.55397049e-03, 4.29337938e-03,
       3.29141858e-03, 2.98520669e-03, 2.34546107e-03, 2.19764267e-03,
       2.14485193e-03, 1.64566905e-03, 1.54347429e-03, 1.64274540e-03,
       1.19037739e-03, 1.24406775e-03, 1.17796275e-03, 1.09351624e-03,
       9.86090211e-04, 9.77755043e-04, 8.91411028e-04, 6.89029204e-04,
       8.88709214e-04, 8.37276120e-04, 6.47759803e-04, 5.95664340e-04,
       5.68601451e-04, 4.42357820e-04, 5.23773632e-04, 4.99593978e-04,
       4.74104661e-04, 4.89202037e-04, 4.73545209e-04, 4.57667899e-04,
       4.18318669e-04, 3.79992709e-04, 3.64570955e-04, 2.49824444e-04,
       3.30237593e-04, 2.83348517e-04, 3.09869222e-04, 2.69232915e-04,
       2.62819959e-04, 2.53630310e-04, 2.30109484e-04, 1.87879735e-04,
       2.16451113e-04, 2.07711793e-04, 1.95861472e-04, 1.90839802e-04,
       1.57123022e-04, 1.75319791e-04, 1.66834444e-04, 1.63264521e-04,
       1.55533359e-04, 1.54318405e-04, 1.45301213e-04, 1.47343100e-04,
      

In [23]:
# tide = utide.reconstruct(time,coef)

In [24]:
# df_all['u_tide'] = tide.u
# df_all['v_tide'] = tide.v
# df_all['u_detide'] = df_all['u'] - df_all['u_tide']
# df_all['v_detide'] = df_all['v'] - df_all['v_tide']

### Resample to hourly values

#### Define date range

In [25]:
# t1 = np.datetime64('2017-11-12T00:00:00')
# t2 = np.datetime64('2018-10-17T00:00:00')
# dff = dfr.loc[(dfr.index >= t1) & (dfr.index <= t2)].interpolate(limit=2)

In [55]:
# resample and interpolate
df_all_r = df_all.resample('1H').mean()

# Calculate tidal currents at hourly points
time_r = mdates.date2num(df_all_r.index)
crit = (coef['diagn']['SNR'] > 2) & (1/coef['aux']['frq'] < 48)
tide_r = utide.reconstruct(time_r,coef,constit=coef['name'][crit])
df_all_r['u_tide'] = tide_r.u
df_all_r['v_tide'] = tide_r.v
df_all_r['u_detide'] = df_all_r['u'] - df_all_r['u_tide']
df_all_r['v_detide'] = df_all_r['v'] - df_all_r['v_tide']

df_all_int = df_all_r.interpolate(limit=int(24*3.5))

df_all_int['u'] = df_all_int['u_detide'] + df_all_int['u_tide']
df_all_int['v'] = df_all_int['v_detide'] + df_all_int['v_tide']

df_all_int['u_f'] = ts.pl64(df_all_int['u_detide'])
df_all_int['v_f'] = ts.pl64(df_all_int['v_detide'])

df_all_int['u_super'] = df_all_int['u'] - df_all_int['u_f'] - df_all_int['u_tide']
df_all_int['v_super'] = df_all_int['v'] - df_all_int['v_f'] - df_all_int['v_tide']

prep/calcs ... done.


/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/scipy/signal/signaltools.py:251: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()


### Subset with no gaps
November 2015-October 2018

In [56]:
# select time range
t1s = np.datetime64('2015-11-11T13:00:00')
t2s = np.datetime64('2018-10-14T07:00:00')
df_sub = df_all_int.loc[(df_all_int.index >= t1s) & (df_all_int.index <= t2s)]

In [57]:
# resample and interpolate
df = df_sub.resample('3H').mean()

### Initial plots

In [58]:
plt.figure(figsize=(7.5,4))
plt.plot(curl_time,curl_tau_F/(beta*H)*8)
plt.plot(df.index,df['u_f'])

FigureCanvasNbAgg()

### Spectral analysis

In [59]:
u = np.array(df['u'])
v = np.array(df['v'])

N = len(u)
fu,Su = welch(u,fs=8.,nperseg=N/3,window='hanning')
fv,Sv = welch(v,fs=8.,nperseg=N/3,window='hanning')

In [60]:
fc,Sc = welch(curl_tau_F[fi]/(beta*H),fs=8.,nperseg=N/3,window='hanning')

In [61]:
from scipy.signal import coherence

fcu,Scu = coherence(u,curl_tau_F[fi]/(beta*H),
                    fs=8.,nperseg=N/3,window='hanning')
fcv,Scv = coherence(v,curl_tau_F[fi]/(beta*H),
                    fs=8.,nperseg=N/3,window='hanning')

In [62]:
fc,Sc = welch(curl_tau_F[fi]/(beta*H),fs=8.,nperseg=N/3,window='hanning')

In [63]:
plt.figure()
plt.semilogx(fcu,Scv,'k-')

FigureCanvasNbAgg()

In [64]:
plt.figure()
plt.loglog(fc,Sc,'k-')

FigureCanvasNbAgg()

In [65]:
plt.figure()
plt.loglog(fu,Su,'k-')
plt.loglog(fv,Sv,'-',color='gray')
xl = plt.xlim()
plt.xlim(xl[0],np.max(fu))

# confidence intervals
dof = 6
edof = (8/3)*dof
lower = edof/chi2.ppf(0.975,edof)
upper = edof/chi2.ppf(0.025,edof)
fplot = 5
fac = 0.0008
plt.plot(np.array([fplot,fplot]),fac*np.array([lower,upper]),'k-')
plt.text(fplot+0.5,fac*np.mean([lower,upper]),'95%',verticalalignment='center')

plt.legend(['u','v'])
plt.xlabel('frequency [cpd]')
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')
plt.title('Benthic Rover\nNov 2015-Oct 2018')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'Benthic Rover\nNov 2015-Oct 2018')

In [66]:
N = len(u)
speed = np.sqrt(u**2 + v**2)
fsp,Ssp = welch(speed,fs=8.,nperseg=N/3,window='hanning')

In [67]:
plt.figure()
plt.loglog(fsp,Ssp,'k-')
xl = plt.xlim()
plt.xlim(xl[0],np.max(fsp))

FigureCanvasNbAgg()

(0.001954529703208945, 3.9985950122936424)

### Lomb-Scargle periodogram - 2014 to 2018

In [68]:
gi, = np.where(np.isfinite(df_all['u']+df_all['v']))
ua = np.array(df_all['u'][gi])
va = np.array(df_all['v'][gi])

#rotary spectrum
fua,ftua,Sua = lombscargle(time[gi],ua,ofac=1,window='hanning',scaling='density')
fva,ftva,Sva = lombscargle(time[gi],va,ofac=1,window='hanning',scaling='density')

In [69]:
# average frequency bands
navg = 2 # number of frequency bands to average
nfreq = int(np.floor(len(fua)/navg))  # number of freqencies in averaged spectrum

fwt = 1/navg
fm = np.zeros(nfreq)
Sum = np.zeros(nfreq)
Svm = np.zeros(nfreq)
for k in range(navg):
    fm = fm + fwt*fua[1:(nfreq*navg)][::navg]
    Sum = Sum + fwt*Sua[1:(nfreq*navg)][::navg]
    Svm = Svm + fwt*Sva[1:(nfreq*navg)][::navg]

In [70]:
Sum

array([3.49997163e-04, 2.22530111e-03, 1.92575183e-03, ...,
       4.81409249e-06, 1.48315243e-07, 2.90089730e-07])

In [71]:
plt.figure()
plt.loglog(fua,Sua)
plt.loglog(fva,Sva,'--')
plt.xlabel('frequency [cpd]')
plt.ylabel('[m$^2$ s$^{-2}$ cpd$^{-1}$]')
plt.title('Lomb-Scargle periodogram [Rover]')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'Lomb-Scargle periodogram [Rover]')

In [72]:
import pycwt as wavelet

In [82]:
fstd = np.std(curl_tau_F[fi]/(beta*H))  # Standard deviation
fvar = fstd ** 2  # Variance
f_norm = curl_tau_F[fi]/(beta*H) / fstd  # Normalized dataset

wstd = np.std(np.squeeze(tauy[fi,ci,ci]))  # Standard deviation
wvar = wstd ** 2  # Variance
w_norm = np.squeeze(tauy[fi,ci,ci]) / wstd  # Normalized dataset

ustd = np.std(u)  # Standard deviation
uvar = ustd ** 2  # Variance
u_norm = u / ustd  # Normalized dataset

vstd = np.std(v)  # Standard deviation
vvar = vstd ** 2  # Variance
v_norm = v / vstd  # Normalized dataset

dt = 1/8 # days

mother = wavelet.Morlet(6)
s0 = 2 * dt  # Starting scale, in this case 2 * 0.25 years = 6 months
dj = 1 / 12  # Twelve sub-octaves per octaves
J = 11 / dj  # Seven powers of two with dj sub-octaves
alphau, _, _ = wavelet.ar1(u_norm)  # Lag-1 autocorrelation for red noise
alphav, _, _ = wavelet.ar1(v_norm)  # Lag-1 autocorrelation for red noise
alphaf, _, _ = wavelet.ar1(f_norm)  # Lag-1 autocorrelation for red noise
alphaw, _, _ = wavelet.ar1(w_norm)  # Lag-1 autocorrelation for red noise

In [83]:
uwave, uscales, ufreqs, ucoi, ufft, ufftfreqs = wavelet.cwt(u_norm, dt, dj, s0, J,
                                                      mother)

vwave, vscales, vfreqs, vcoi, vfft, vfftfreqs = wavelet.cwt(v_norm, dt, dj, s0, J,
                                                      mother)

fwave, fscales, ffreqs, fcoi, ffft, ffftfreqs = wavelet.cwt(f_norm, dt, dj, s0, J,
                                                      mother)

wwave, wscales, wfreqs, wcoi, wfft, wfftfreqs = wavelet.cwt(w_norm, dt, dj, s0, J,
                                                      mother)

In [84]:
uiwave = wavelet.icwt(uwave, uscales, dt, dj, mother) * ustd
viwave = wavelet.icwt(vwave, vscales, dt, dj, mother) * vstd
fiwave = wavelet.icwt(fwave, fscales, dt, dj, mother) * fstd
wiwave = wavelet.icwt(wwave, fscales, dt, dj, mother) * wstd

In [85]:
upower = (np.abs(uwave)) ** 2
ufft_power = np.abs(ufft) ** 2
uperiod = 1 / ufreqs

usignif, ufft_theor = wavelet.significance(1.0, dt, uscales, 0, alphau,
                                         significance_level=0.99,
                                         wavelet=mother)
usig99 = np.ones([1, N]) * usignif[:, None]
usig99 = upower / usig99

vpower = (np.abs(vwave)) ** 2
vfft_power = np.abs(vfft) ** 2
vperiod = 1 / vfreqs

vsignif, vfft_theor = wavelet.significance(1.0, dt, vscales, 0, alphav,
                                         significance_level=0.99,
                                         wavelet=mother)
vsig99 = np.ones([1, N]) * vsignif[:, None]
vsig99 = vpower / vsig99

fpower = (np.abs(fwave)) ** 2
ffft_power = np.abs(ffft) ** 2
fperiod = 1 / ffreqs

fsignif, ffft_theor = wavelet.significance(1.0, dt, fscales, 0, alphaf,
                                         significance_level=0.99,
                                         wavelet=mother)
fsig99 = np.ones([1, N]) * fsignif[:, None]
fsig99 = fpower / fsig99

wpower = (np.abs(wwave)) ** 2
wfft_power = np.abs(wfft) ** 2
wperiod = 1 / wfreqs

wsignif, wfft_theor = wavelet.significance(1.0, dt, wscales, 0, alphaw,
                                         significance_level=0.99,
                                         wavelet=mother)
wsig99 = np.ones([1, N]) * wsignif[:, None]
wsig99 = wpower / wsig99

In [92]:
time = curl_time[fi]
t = np.array(curl_time[fi])

plt.figure(figsize=(6,8))

ax1 = plt.subplot(311)
plt.contourf(time, np.log2(uperiod), upower,
            extend='both',cmap='RdPu')
plt.colorbar()
plt.contour(time, np.log2(uperiod), usig99, [-99, 1], colors='k', linewidths=1)
yticks = 2 ** np.arange(np.ceil(np.log2(uperiod.min())),
                           np.ceil(np.log2(uperiod.max())))
ax1.set_yticks(np.log2(yticks))
ax1.set_yticklabels(yticks)
#ax1.set_xticklabels([])
plt.ylim([np.log2(2),np.log2(uperiod.max())])
plt.fill(np.concatenate([t, t[-1:] + dt, t[-1:] + dt,
                           t[:1] - dt, t[:1] - dt]),
        np.concatenate([np.log2(ucoi), [1e-9], np.log2(uperiod[-1:]),
                           np.log2(uperiod[-1:]), [1e-9]]),
        'k', alpha=0.5, hatch='x')
plt.xticks(rotation=30)
plt.ylabel('period [d]')
plt.title('wavelet power\nRover eastward velocity - $u$')

ax2 = plt.subplot(312)
plt.contourf(time, np.log2(vperiod), vpower,
            extend='both',cmap='RdPu')
plt.colorbar()
plt.contour(time, np.log2(vperiod), vsig99, [-99, 1], colors='k', linewidths=1)
yticks = 2 ** np.arange(np.ceil(np.log2(vperiod.min())),
                           np.ceil(np.log2(vperiod.max())))
ax2.set_yticks(np.log2(yticks))
ax2.set_yticklabels(yticks)
ax2.set_xticklabels([])
plt.ylim([np.log2(2),np.log2(period.max())])
plt.fill(np.concatenate([t, t[-1:] + dt, t[-1:] + dt,
                           t[:1] - dt, t[:1] - dt]),
        np.concatenate([np.log2(vcoi), [1e-9], np.log2(vperiod[-1:]),
                           np.log2(vperiod[-1:]), [1e-9]]),
        'k', alpha=0.5, hatch='x')
plt.xticks(rotation=30)
plt.ylabel('period [d]')
plt.title('Rover northward velocity - $v$')

ax3 = plt.subplot(313)
plt.contourf(time, np.log2(fperiod), fpower,
            extend='both',cmap='RdPu')
plt.colorbar()
plt.contour(time, np.log2(fperiod), fsig99, [-99, 1], colors='k', linewidths=1)
yticks = 2 ** np.arange(np.ceil(np.log2(vperiod.min())),
                           np.ceil(np.log2(vperiod.max())))
ax3.set_yticks(np.log2(yticks))
ax3.set_yticklabels(yticks)
plt.ylim([np.log2(2),np.log2(period.max())])
plt.fill(np.concatenate([t, t[-1:] + dt, t[-1:] + dt,
                           t[:1] - dt, t[:1] - dt]),
        np.concatenate([np.log2(fcoi), [1e-9], np.log2(fperiod[-1:]),
                           np.log2(fperiod[-1:]), [1e-9]]),
        'k', alpha=0.5, hatch='x')
plt.xticks(rotation=30)
plt.ylabel('period [d]')
plt.title('NCEP-NARR wind forcing - $F$')
plt.tight_layout()

plt.savefig('figures_paper/wavelet_power.png',dpi=1000)
plt.savefig('figures_paper/wavelet_power.pdf')

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

TypeError: ufunc add cannot use operands with types dtype('<M8[ns]') and dtype('float64')

In [ ]:
plt.figure()
plt.contourf(time, np.log2(wperiod), wpower,
            extend='both',cmap='RdPu')
plt.colorbar()
plt.contour(time, np.log2(wperiod), wsig99, [-99, 1], colors='k', linewidths=1)
yticks = 2 ** np.arange(np.ceil(np.log2(wperiod.min())),
                           np.ceil(np.log2(wperiod.max())))
plt.gca().set_yticks(np.log2(yticks))
plt.gca().set_yticklabels(yticks)
plt.ylim([np.log2(2),np.log2(wperiod.max())])
plt.fill(np.concatenate([t, t[-1:] + dt, t[-1:] + dt,
                           t[:1] - dt, t[:1] - dt]),
        np.concatenate([np.log2(wcoi), [1e-9], np.log2(wperiod[-1:]),
                           np.log2(wperiod[-1:]), [1e-9]]),
        'k', alpha=0.5, hatch='x')
plt.xticks(rotation=30)
plt.tight_layout()

In [ ]:
xwt, xcoi, xfft, xfreqs = wavelet.xwt(u_norm, f_norm, dt, dj, s0, J,wavelet=mother)

In [ ]:
#xwct, xwcoi, xwfft, xwfreqs = wavelet.wct(u_norm, f_norm, dt, dj, s0, int(J),cache=True)

In [ ]:
plt.figure()
plt.contourf(t, np.log2(uperiod), np.abs(xwt)**2,
            extend='both',cmap='RdPu')
plt.colorbar()
#plt.contour(t, np.log2(uperiod), usig99, [-99, 1], colors='k', linewidths=1)

plt.fill(np.concatenate([t, t[-1:] + dt, t[-1:] + dt,
                           t[:1] - dt, t[:1] - dt]),
        np.concatenate([np.log2(xcoi), [1e-9], np.log2(uperiod[-1:]),
                           np.log2(uperiod[-1:]), [1e-9]]),
        'k', alpha=0.5, hatch='x')

In [ ]:
help(wavelet.wct)

In [ ]:
J

In [ ]:
dir(wavelet)